In [ ]:
!pip install safetensors


In [ ]:
!pip install transformers datasets scikit-learn

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/yorumlar/isim_yorum_kategorili_gelistirilmis.csv')

df = df.dropna(subset=['yorum', 'etiket'])
df['etiket'] = df['etiket'].astype(int)

X = df['yorum'].tolist()
y = df['etiket'].tolist()


In [ ]:
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

def encode(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

from datasets import Dataset
dataset = Dataset.from_dict({'text': X, 'label': y})
dataset = dataset.map(encode, batched=True) # Added batched=True
dataset = dataset.train_test_split(test_size=0.2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

Map:   0%|          | 0/518 [00:00<?, ? examples/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-cased", num_labels=2)

training_args = TrainingArguments(
    output_dir='./sikayet_eğitim_modeli',
    eval_strategy="epoch", # Changed from evaluation_strategy
    save_strategy="epoch",
    num_train_epochs=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir='./logs',
)

def compute_metrics(p):
    from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
    preds = np.argmax(p.predictions, axis=1)
    return {
        'accuracy': accuracy_score(p.label_ids, preds),
        'precision': precision_score(p.label_ids, preds),
        'recall': recall_score(p.label_ids, preds),
        'f1': f1_score(p.label_ids, preds)
    }

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: akbulutseymanur57 (akbulutseymanur57-nstitution) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.466800,0.097682,0.961538,0.970588,0.970588,0.970588
2,0.037300,0.115963,0.961538,0.970588,0.970588,0.970588
3,0.042400,0.047012,0.990385,1.000000,0.985294,0.992593
4,0.023700,0.047013,0.990385,1.000000,0.985294,0.992593
5,0.000300,0.011099,0.990385,1.000000,0.985294,0.992593
6,0.000100,0.016467,0.990385,1.000000,0.985294,0.992593
7,0.000100,0.016224,0.990385,1.000000,0.985294,0.992593
8,0.000100,0.016245,0.990385,1.000000,0.985294,0.992593
9,0.000100,0.017441,0.990385,1.000000,0.985294,0.992593
10,0.000100,0.017480,0.990385,1.000000,0.985294,0.992593


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packa

TrainOutput(global_step=2600, training_loss=0.005719480719954635, metrics={'train_runtime': 1201.3986, 'train_samples_per_second': 34.46, 'train_steps_per_second': 2.164, 'total_flos': 2723199422976000.0, 'train_loss': 0.005719480719954635, 'epoch': 100.0})

In [ ]:
model.save_pretrained('/content/drive/MyDrive/yorumlar/sikayet_eğitim_modeli')
tokenizer.save_pretrained('/content/drive/MyDrive/yorumlar/sikayet_eğitim_modeli')


('/content/drive/MyDrive/yorumlar/sikayet_eğitim_modeli/tokenizer_config.json',
 '/content/drive/MyDrive/yorumlar/sikayet_eğitim_modeli/special_tokens_map.json',
 '/content/drive/MyDrive/yorumlar/sikayet_eğitim_modeli/vocab.txt',
 '/content/drive/MyDrive/yorumlar/sikayet_eğitim_modeli/added_tokens.json')